In [2]:
import pandasdmx as sdmx
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
# from jupyter_dash import JupyterDash

In [106]:
ind_code = "HT_ADOL_UNMETMED_NOUNMET"

unicef = sdmx.Request("UNICEF")

metadata = unicef.dataflow("TRANSMONEE", provider="ECARO", version="1.0")
metadata

<pandasdmx.StructureMessage>
  <Header>
    id: 'IREF152663'
    prepared: '2022-02-16T12:41:02+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency UNICEF>
    source: 
    test: False
  response: <Response [200]>
  Codelist (37): CL_INDICATORS_TRANSMONEE CL_CONF_STATUS CL_DECIMALS CL...
  ConceptScheme (3): CROSS_DOMAIN_CONCEPTS CS_SDG UNICEF_CONCEPTS
  DataflowDefinition (1): TRANSMONEE
  DataStructureDefinition (1): DSD_ECARO_TRANSMONEE
  AgencyScheme (2): AGENCIES DATA_PROVIDERS
  ProvisionAgreement (1): TRANSMONEE_ECARO_ECARO

In [22]:
metadata.response.url

'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/dataflow/ECARO/TRANSMONEE/1.0?references=all'

In [23]:
metadata.response.from_cache

False

In [6]:
metadata

<pandasdmx.StructureMessage>
  <Header>
    id: 'IREF578808'
    prepared: '2022-02-15T20:44:45+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency UNICEF>
    source: 
    test: False
  response: <Response [200]>
  Codelist (37): CL_INDICATORS_TRANSMONEE CL_CONF_STATUS CL_DECIMALS CL...
  ConceptScheme (3): CROSS_DOMAIN_CONCEPTS CS_SDG UNICEF_CONCEPTS
  DataflowDefinition (1): TRANSMONEE
  DataStructureDefinition (1): DSD_ECARO_TRANSMONEE
  AgencyScheme (2): AGENCIES DATA_PROVIDERS
  ProvisionAgreement (1): TRANSMONEE_ECARO_ECARO

In [7]:
metadata.response.url

'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/dataflow/ECARO/TRANSMONEE/1.0?references=all'

In [24]:
dataflows = sdmx.to_pandas(metadata.dataflow)
dataflows

TRANSMONEE    ECARO for TransMonEE
dtype: object

In [159]:
cl_age = unicef.codelist("CL_AGE", version="1.0")
age_groups = sdmx.to_pandas(cl_age)
dict_age_groups = age_groups['codelist']['CL_AGE'].reset_index()
dict_age_groups


,CL_AGE,name,parent
0,M0,Under 1 month old,CL_AGE
1,M0T1,Under 2 months old,CL_AGE
2,M0T11,Under 12 months old,CL_AGE
3,M0T23,Under 24 months old,CL_AGE
4,M0T35,Under 36 months old,CL_AGE
...,...,...,...
389,Y_GE85,85 years old and over,CL_AGE
390,Y_GE90,90 years old and over,CL_AGE
391,Y_GE95,95 years old and over,CL_AGE
392,_T,Total,CL_AGE


In [103]:
age_groups = sdmx.to_pandas(metadata.codelist['CL_AGE'])
age_groups

""
CL_AGE


In [94]:
gender = list(dsd.dimensions.get('RESIDENCE').local_representation.enumerated)[0]
gender.id


'_T'

In [12]:
dsd.attributes.components

[<DataAttribute COVERAGE_TIME>,
 <DataAttribute UNIT_MEASURE>,
 <DataAttribute OBS_FOOTNOTE>,
 <DataAttribute FREQ>,
 <DataAttribute DATA_SOURCE>,
 <DataAttribute UNIT_MULTIPLIER>,
 <DataAttribute OBS_STATUS>]

In [81]:
list(dsd.attributes.get('UNIT_MEASURE').local_representation.enumerated)

[<Code BINARY: Binary>,
 <Code BIRTHS: Births>,
 <Code BIRTHSPER1000WOMEN: Births per 1000 women>,
 <Code BIRTHS_1000: 1000 Births>,
 <Code D: Number of deaths>,
 <Code DPIA: Adjusted disability parity index>,
 <Code D_PER_1000_1: Deaths per 1000 children aged 1>,
 <Code D_PER_1000_10: Deaths per 1000 children aged 10>,
 <Code D_PER_1000_15: Deaths per 1000 children aged 15>,
 <Code D_PER_1000_20: Deaths per 1000 children aged 20>,
 <Code D_PER_1000_5: Deaths per 1000 children aged 5>,
 <Code D_PER_1000_B: Deaths per 1000 live births>,
 <Code EUR: Euro>,
 <Code GDP_CAP: As % of GDP per capita>,
 <Code GDP_PERC: % of GDP>,
 <Code GOV_EXP_EDU: % of government expenditure on education>,
 <Code GOV_EXP_HT: Percentage of government expenditure on health>,
 <Code GOV_EXP_T: % of total government expenditure>,
 <Code GPERM3: Micrograms per cubic meter>,
 <Code GPI: Gender Parity Index>,
 <Code GPIA: Adjusted gender parity index>,
 <Code HRS_WK: Hours Worked>,
 <Code IDX: Index>,
 <Code LIVEBI

In [52]:
units=list(dsd.attributes.get('UNIT_MEASURE').local_representation.enumerated)
len(units)

61

In [83]:
unit=list(dsd.attributes.get('UNIT_MEASURE').local_representation.enumerated)[2]
unit.id

'BIRTHSPER1000WOMEN'

In [13]:
dsd.measures.components

[<PrimaryMeasure OBS_VALUE>]

In [27]:
dsd.dimensions

<DimensionDescriptor: <Dimension REF_AREA>; <Dimension INDICATOR>; <Dimension SEX>; <Dimension AGE>; <Dimension RESIDENCE>; <Dimension WEALTH_QUINTILE>; <TimeDimension TIME_PERIOD>>

In [9]:
DEFAULT_DIMENSIONS = [
    "SEX",
    "RESIDENCE",
    "WEALTH_QUINTILE",
    "AGE",
    "TOTAL"
]

In [3]:
import json
with open("./transmonee_dashboard/src/transmonee_dashboard/pages/indicator_config.json") as config_file:
    indicators_config = json.load(config_file)

In [19]:
indicators = ["CME_MRY0"]
keys = dict(INDICATOR=indicators)
dimensions = {"SEX": ["F"]}
keys.update(dimensions)
for indicator in indicators:
    indicator_config = indicators_config.get(indicator, {})
    print(indicator_config)
    for dim in dimensions.keys():
        # if the dimension is in the config and has not overridden here
        if (dim in keys and dim in indicator_config) and not keys[dim]:
            # replace the empty with the config value
            keys.pop(dim)
            keys.update(indicator_config[dim])
    
keys

{'SEX': {'SEX': ['F', 'M']}, 'TOTAL': {'SEX': ['_T']}}


{'INDICATOR': ['CME_MRY0'], 'SEX': ['F']}

In [17]:
message = unicef.data(
    "TRANSMONEE", 
    provider="ECARO", 
    key={
        'INDICATOR': ['EDUNF_ADMIN_L2_MAT'],
        'SEX': ['_T'],
        'RESIDENCE': ['_T'],
        'WEALTH_QUINTILE': ['_T']
    }, 
    params = dict(startPeriod='2010', endPeriod='2021', lastNObservations=1), 
    dsd=dsd
)

/usr/local/lib/python3.9/site-packages/pandasdmx/api.py:260: UserWarning: 'provider' argument is redundant for <Resource.data: 'data'>
  warn(f"'provider' argument is redundant for {resource_type!r}")


In [18]:
message.response.from_cache

False

In [19]:
message.response.url

'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/TRANSMONEE/.EDUNF_OFST_L1+EDUNF_OFST_L2+EDUNF_OFST_L3.F+M.._T._T?startPeriod=2010&endPeriod=2021&lastNObservations=1&labels=True'

In [20]:
https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/TRANSMONEE/ALB+AND+ARM+AUT+AZE+BEL+BGR+BIH+BLR+CHE+CYP+CZE+DEU+DNK+ESP+EST+FIN+FRA+GBR+GEO+GRC+HRV+HUN+IRL+ISL+ITA+KAZ+KGZ+LIE+LTU+LUX+LVA+MCO+MDA+MKD+MLT+MNE+NLD+NOR+POL+PRT+ROU+RUS+SMR+SRB+SVK+SVN+SWE+TJK+TKM+TUR+UKR+UZB+VAT+XKX.EDUNF_OFST_L1+EDUNF_OFST_L2+EDUNF_OFST_L3._T._T._T._T?startPeriod=2010&endPeriod=2020&lastNObservations=1 

SyntaxError: invalid syntax (<ipython-input-20-f51f2b5b6cdf>, line 1)

In [ ]:
df = message.to_pandas(attributes="o", rtype="rows", dtype="str")

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.columns.names

FrozenList([None])

In [ ]:
df.columns.levels

AttributeError: 'Index' object has no attribute 'levels'

In [ ]:
df.index

DatetimeIndex(['2010-01-01', '2011-01-01'], dtype='datetime64[ns]', name='TIME_PERIOD', freq=None)

In [ ]:
len(df.columns.levels)

6

In [ ]:
df.columns = df.columns.droplevel(-1)

In [ ]:
df

,,,,,,,value,UNIT_MEASURE,FREQ,DATA_SOURCE,UNIT_MULTIPLIER,OBS_STATUS
REF_AREA,INDICATOR,SEX,AGE,RESIDENCE,WEALTH_QUINTILE,TIME_PERIOD,,,,,,
AND,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,No,BINARY,1,UNESCO,0,A
ALB,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
ARM,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
AUT,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
AZE,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,No,BINARY,1,UNESCO,0,A
BIH,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
BEL,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
BGR,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
BLR,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A


In [ ]:
df["AGE"].unique()

array(['OVER_AGE', 'SCHOOL_AGE', 'UNDER_AGE'], dtype=object)

In [ ]:
df.xs(('_T', '_T'), level=('SEX', 'RESIDENCE'), axis=1, drop_level=False)

REF_AREA                             AUT                                     \
INDICATOR       HT_ADOL_UNMETMED_NOUNMET                                      
SEX                                   _T                                      
AGE                               Y16T19 Y16T24 Y16T29 Y16T44 Y16T64 Y18T44   
RESIDENCE                             _T     _T     _T     _T     _T     _T   
WEALTH_QUINTILE                       _T     _T     _T     _T     _T     _T   
TIME_PERIOD                                                                   
2010-01-01                          96.3   97.0   97.2   97.2   97.4   97.1   
2011-01-01                          98.3   98.4   98.4   97.9   97.9   97.9   

REF_AREA                                     ...                      TUR  \
INDICATOR                                    ... HT_ADOL_UNMETMED_NOUNMET   
SEX                                          ...                       _T   
AGE             Y20T24 Y20T29 Y25T29 Y25T34  ...                   Y35T44   
RESIDENCE           _T     _T     _T     _T  ...                       _T   
WEALTH_QUINTILE     _T     _T     _T     _T  ...                       _T   
TIME_PERIOD                                  ...                            
2010-01-01        97.4   97.6   97.7   97.6  ...                     72.9   
2011-01-01        98.4   98.4   98.4   97.7  ...                     77.9   

REF_AREA                                                                        
INDICATOR                                                                       
SEX                                                                             
AGE             Y45T54 Y45T64 Y55T64 Y65T74 Y75T84 Y_GE16 Y_GE65 Y_GE75 Y_GE85  
RESIDENCE           _T     _T     _T     _T     _T     _T     _T     _T     _T  
WEALTH_QUINTILE     _T     _T     _T     _T     _T     _T     _T     _T     _T  
TIME_PERIOD                                                                     
2010-01-01        74.4   75.9   78.1   77.5   79.2   78.3   77.7   78.1   68.6  
2011-01-01        80.5   81.3   82.6   80.9   80.2   82.9   80.4   79.6   75.8  

[2 rows x 658 columns]

In [ ]:
df.columns = df.columns.droplevel(-1)

In [ ]:
df

REF_AREA                         AUT                                     \
INDICATOR   HT_ADOL_UNMETMED_NOUNMET                                      
SEX                               _T                                      
AGE                           Y16T19 Y16T24 Y16T29 Y16T44 Y16T64 Y18T44   
RESIDENCE                         _T     _T     _T     _T     _T     _T   
TIME_PERIOD                                                               
2010-01-01                      96.3   97.0   97.2   97.2   97.4   97.1   
2011-01-01                      98.3   98.4   98.4   97.9   97.9   97.9   

REF_AREA                                 ...                      TUR         \
INDICATOR                                ... HT_ADOL_UNMETMED_NOUNMET          
SEX                                      ...                       _T          
AGE         Y20T24 Y20T29 Y25T29 Y25T34  ...                   Y35T44 Y45T54   
RESIDENCE       _T     _T     _T     _T  ...                       _T     _T   
TIME_PERIOD                              ...                                   
2010-01-01    97.4   97.6   97.7   97.6  ...                     72.9   74.4   
2011-01-01    98.4   98.4   98.4   97.7  ...                     77.9   80.5   

REF_AREA                                                             
INDICATOR                                                            
SEX                                                                  
AGE         Y45T64 Y55T64 Y65T74 Y75T84 Y_GE16 Y_GE65 Y_GE75 Y_GE85  
RESIDENCE       _T     _T     _T     _T     _T     _T     _T     _T  
TIME_PERIOD                                                          
2010-01-01    75.9   78.1   77.5   79.2   78.3   77.7   78.1   68.6  
2011-01-01    81.3   82.6   80.9   80.2   82.9   80.4   79.6   75.8  

[2 rows x 658 columns]

In [ ]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Graph(id='graph', figure=px.bar(df, x='REF_AREA', y='value', color='SEX', barmode='group', title='Indicator')),
    
])
# # Define callback to update graph
# @app.callback(
#     Output('graph', 'figure'),
#     [Input("colorscale-dropdown", "value")]
# )
# def update_figure(colorscale):
#     return px.scatter(
#         df, x="total_bill", y="tip", color="size",
#         color_continuous_scale=colorscale,
#         render_mode="webgl", title="Tips"
#     )
# Run app and display result inline in the notebook
del app.config._read_only["requests_pathname_prefix"]
app.run_server(mode='inline')

/opt/homebrew/Caskroom/miniforge/base/envs/dash/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

